This notebook was prepared by [Donne Martin](https://github.com/donnemartin). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Challenge Notebook

## Problem: Implement a trie with find, insert, remove, and list_words methods.

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Can we assume we are working with strings?
    * Yes
* Are the strings in ASCII?
    * Yes
* Should `find` only match exact words with a terminating character?
    * Yes
* Should `list_words` only return words with a terminating character?
    * Yes
* Can we assume this fits memory?
    * Yes

## Test Cases

<pre>

         root
       /  |  \
      h   a*  m
     / \   \   \
    a   e*  t*  e*
   / \         / \
  s*  t*      n*  t*
             /
            s*

find

* Find on an empty trie
* Find non-matching
* Find matching

insert

* Insert on empty trie
* Insert to make a leaf terminator char
* Insert to extend an existing terminator char

remove

* Remove me
* Remove mens
* Remove a
* Remove has

list_words

* List empty
* List general case
</pre>



## Algorithm

Refer to the [Solution Notebook](http://nbviewer.ipython.org/github/donnemartin/interactive-coding-challenges/blob/master/graphs_trees/trie/trie_solution.ipynb).  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.

### find

* Set node to the root
* For each char in the input word
    * Check the current node's children to see if it contains the char
        * If a child has the char, set node to the child
        * Else, return None
* Return the last child node if it has a terminator, else None

Complexity:
* Time: O(m), where m is the length of the word
* Space: O(h) for the recursion depth (tree height), or O(1) if using an iterative approach

### insert

* set node to the root
* For each char in the input word
    * Check the current node's children to see if it contains the char
        * If a child has the char, set node to the child
        * Else, insert a new node with the char
            * Update children and parents
* Set the last node as a terminating node

Complexity:
* Time: O(m), where m is the length of the word
* Space: O(h) for the recursion depth (tree height), or O(1) if using an iterative approach

### remove

* Determine the matching terminating node by calling the find method
* If the matching node has children, remove the terminator to prevent orphaning its children
* Set the parent node to the matching node's parent
* We'll be looping up the parent chain to propagate the delete
* While the parent is valid
    * If the node has children
        * Return to prevent orphaning its remaining children
    * If the node is a terminating node and it isn't the original matching node from the find call
        * Return to prevent deleting this additional valid word
    * Remove the parent node's child entry matching the node
    * Set the node to the parent
    * Set the parent to the parent's parent

Complexity:
* Time: O(m+h), where where m is the length of the word and h is the tree height
* Space: O(h) for the recursion depth (tree height), or O(1) if using an iterative approach

### list_words

* Do a pre-order traversal, passing down the current word
    * When you reach a terminating node, add it to the list of results

Complexity:
* Time: O(n)
* Space: O(h) for the recursion depth (tree height), or O(1) if using an iterative approach

## Code

In [97]:
trie = Trie()

print('Test: Insert')
words = ['a', 'at', 'has', 'hat', 'he',
         'me', 'men', 'mens', 'met']
for word in words:
    trie.insert(word)

for word in trie.list_words():
    assert_true(trie.find(word) is not None)

trie.insert('wtf')
trie.insert('bbq')
trie.list_words()

Test: Insert


['a', 'at', 'has', 'hat', 'he', 'bbq', 'me', 'met', 'men', 'mens', 'wtf']

In [99]:
trie.remove('bbq')

In [93]:
from collections import OrderedDict

class Node(object):
    def __init__(self, data, parent=None, terminates=False):
        if data is None:
            raise TypeError('data cannot be none.')
        self.data = data
        self.terminates = terminates
        self.parent = parent
        self.children = {}
        
class Trie(object):

    def __init__(self):
        self.root = Node('')

    def find(self, word):
        if word is None:
            raise TypeError('Words cannot be blank or none.')
        node = self.root
        
        # traverse down the trie path
        for char in word:
            if char in node.children.keys():
                node = node.children[char]
            else:
                return None
            
        return node if node.terminates else None

    def insert(self, word):
        if word is None:
            raise TypeError('Words cannot be blank or none.')
            
        node = self.root
        parent = None
        
        # go through each char
        for char in word:
            # traverse down the node if it already exists
            if char in node.children:
                node = node.children[char]
            # add the node, then traverse down
            else:
                node.children[char] = Node(char, parent = node)
                node = node.children[char]
        
        node.terminates = True #last char terminates the word
        

    def remove(self, word):
        if word is None:
            raise TypeError('Word cannot be none.')
        node = self.find(word)
        
        if node is None:
            return
        # reset it to a None-terminating node
        node.terminates = False
        
        parent = node.parent
        # prevent the delete from deleting nodes that share the same parent (its siblings) 
        # or terminating nodes of other words
        while parent is not None:
            if node.children or node.terminates:
                return
            del parent.children[node.data]
            node = parent
            parent = parent.parent                 
    
    def list_words(self):
        results = []
        curr_word = ''
        self._list_words(self.root, curr_word, results)
        return results
    
    # recursively explore all paths down the TRIE
    def _list_words(self, node, curr_word, results):
        if node is None:
            return
        for key, child in node.children.items():
            if child.terminates:
                results.append(curr_word + key)
            self._list_words(child, curr_word + key, results)        

## Unit Test

**The following unit test is expected to fail until you solve the challenge.**

In [94]:
# %load test_trie.py
from nose.tools import assert_true


class TestTrie(object):

    def test_trie(self):
        print('Test: Remove from empty trie')
        trie = Trie()
        assert_true(trie.remove('foo') is None)

        print('Test: Insert')
        words = ['a', 'at', 'has', 'hat', 'he',
                 'me', 'men', 'mens', 'met']
        for word in words:
            trie.insert(word)
        for word in trie.list_words():
            assert_true(trie.find(word) is not None)

        # Remove me
        # Remove mens
        # Remove a
            
        print('Test: Remove me')
        trie.remove('me')
        words_removed = ['me']
        words = ['a', 'at', 'has', 'hat', 'he',
                 'men', 'mens', 'met']
        for word in words:
            assert_true(trie.find(word) is not None)
        for word in words_removed:
            assert_true(trie.find(word) is None)

        print('Test: Remove mens')
        trie.remove('mens')
        words_removed = ['me', 'mens']
        words = ['a', 'at', 'has', 'hat', 'he',
                 'men', 'met']
        for word in words:
            assert_true(trie.find(word) is not None)
        for word in words_removed:
            assert_true(trie.find(word) is None)

        print('Test: Remove a')
        trie.remove('a')
        words_removed = ['a', 'me', 'mens']
        words = ['at', 'has', 'hat', 'he',
                 'men', 'met']
        for word in words:
            assert_true(trie.find(word) is not None)
        for word in words_removed:
            assert_true(trie.find(word) is None)

        print('Test: Remove has')
        trie.remove('has')
        words_removed = ['a', 'has', 'me', 'mens']
        words = ['at', 'hat', 'he',
                 'men', 'met']
        for word in words:
            assert_true(trie.find(word) is not None)
        for word in words_removed:
            assert_true(trie.find(word) is None)

        print('Success: test_trie')


def main():
    test = TestTrie()
    test.test_trie()


if __name__ == '__main__':
    main()

Test: Remove from empty trie
Test: Insert
Test: Remove me
Test: Remove mens
Test: Remove a
Test: Remove has
Success: test_trie


## Solution Notebook

Review the [Solution Notebook](http://nbviewer.ipython.org/github/donnemartin/interactive-coding-challenges/blob/master/graphs_trees/trie/trie_solution.ipynb) for a discussion on algorithms and code solutions.